## Processing Ocean Color Imagery - Process Micasense Images

Pull in all images and process with micasense tools
    normalize and align and whatnot
    put these into functions and clean up
    also visualize along the way and look at distributions to think about whether or not they're legitimate
    write out
    
Pull in all images as modified and metadata with specific functions
    Visualize for issues and shadows and bright spots and locations
    correct for sky radiance
    visualize the distributions of both sky (clouds) and sea
    look at the variability of spectra both for specific pixels and across nearby images
    filter for orientation and location and issues

take the water leaving spectra and convert to chla as well as other future products and just intro general analysis of spectra
    produce a csv or .shp or images of ocean color products 
    

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ipywidgets import FloatProgress, Layout
from IPython.display import display
from micasense import imageset as imageset
from micasense import capture as capture
import multiprocessing
import glob, shutil, os

import cv2
import numpy as np
import matplotlib.pyplot as plt


import micasense.imageutils as imageutils
import micasense.plotutils as plotutils

import exiftool
import datetime

import subprocess
import math


### Run the process

#### first sort the images into sea and sky

In [13]:
! ls data/20210407_duml_full_workflow_test

all_imgs  panel_rededge  sea_rededge  sky_rededge
panel	  sea		 sky	      sunblocked_rededge


In [15]:
all_img_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'all_imgs')
sky_dst = os.path.join('data', '20210407_duml_full_workflow_test', 'sky')
sea_dst = os.path.join('data', '20210407_duml_full_workflow_test', 'sea')

In [16]:
if not os.path.exists(sky_dst):
    os.makedirs(sky_dst)

if not os.path.exists(sea_dst):
    os.makedirs(sea_dst)

down_image_count = 10
up_image_count = 8

#go through each image
i = 0
j = 0
for file in sorted(glob.glob(all_img_dir+ "/*.tif")):
    if i < down_image_count*6:
        shutil.move(file, sea_dst)
        i += 1
        if i == down_image_count*6:
            j = 0
    elif j < up_image_count*6:
        shutil.move(file, sky_dst)
        j += 1
        if j == up_image_count*6:
            i = 0

#### Now the real processing code

In [17]:
! ls data/20210407_duml_full_workflow_test

all_imgs  panel_rededge  sea_rededge  sky_rededge
panel	  sea		 sky	      sunblocked_rededge


In [20]:
glob.glob(os.path.join('data', '20210317_duml_360_altitude_testing', 'altitude_test', 'panel', 'IMG_0495*.tif'))

['data/20210317_duml_360_altitude_testing/altitude_test/panel/IMG_0495_2.tif',
 'data/20210317_duml_360_altitude_testing/altitude_test/panel/IMG_0495_5.tif',
 'data/20210317_duml_360_altitude_testing/altitude_test/panel/IMG_0495_3.tif',
 'data/20210317_duml_360_altitude_testing/altitude_test/panel/IMG_0495_1.tif',
 'data/20210317_duml_360_altitude_testing/altitude_test/panel/IMG_0495_6.tif',
 'data/20210317_duml_360_altitude_testing/altitude_test/panel/IMG_0495_4.tif']

In [45]:
warp_img_dir

'data/20210317_duml_360_altitude_testing/varying_altitude'

In [51]:
panelNames = None
#panelNames = glob.glob(os.path.join('data', '20210317_duml_360_altitude_testing', 'altitude_test', 'panel', 'IMG_0494_*.tif'))
alt_dir = os.path.join('data', '20210317_duml_360_altitude_testing', 'altitude_test', 'varying_altitude')

warp_img_dir = os.path.join('data', '20210317_duml_360_altitude_testing', 'altitude_test', 'varying_altitude')

altitude_out = process_micasense_subset(alt_dir, panelNames, warp_img_dir, overwrite=False)

Not using panel irradiance.
used warp dir data/20210317_duml_360_altitude_testing/altitude_test/varying_altitude
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 2
Finished aligning band 1
Finished aligning band 0
Finished aligning band 3
Finished aligning band 4
Saving time: 0:17:05.402282
Alignment+Saving rate: 0.33 images per second
Finished saving images.
exiftool -csv="data/20210317_duml_360_altitude_testing/altitude_test/varying_altitude/stacks/log.csv" -overwrite_original data/20210317_duml_360_altitude_testing/altitude_test/varying_altitude/stacks
Finished saving image metadata.


In [ ]:
panelNames = glob.glob(os.path.join('data', '20210317_duml_360_altitude_testing', 'altitude_test', 'panel', 'IMG_0494_*.tif'))
alt_dir = os.path.join('data', '20210317_duml_360_altitude_testing', 'altitude_test', 'varying_altitude')

warp_img_dir = os.path.join('data', '20210317_duml_360_altitude_testing', 'panel')

altitude_out = process_micasense_subset(alt_dir, panelNames, warp_img_dir, overwrite=False, panelCorners=[[150, 500], [150, 1000], [600, 1000], [600, 500]])

### This is the typical full workflow

In [21]:
glob.glob(os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'panel', 'IMG_0008*.tif'))

['data/20210407_duml_full_workflow_test/panel/IMG_0008_1.tif',
 'data/20210407_duml_full_workflow_test/panel/IMG_0008_5.tif',
 'data/20210407_duml_full_workflow_test/panel/IMG_0008_2.tif',
 'data/20210407_duml_full_workflow_test/panel/IMG_0008_3.tif',
 'data/20210407_duml_full_workflow_test/panel/IMG_0008_6.tif',
 'data/20210407_duml_full_workflow_test/panel/IMG_0008_4.tif']

In [22]:
! ls data/20210407_duml_full_workflow_test/

all_imgs  panel_rededge  sea_rededge  sky_rededge
panel	  sea		 sky	      sunblocked_rededge


First process the Altum images

In [32]:
c = capture.Capture.from_filelist(panelNames)

In [35]:
imgset = imageset.ImageSet.from_directory(os.path.join('data', '20210407_duml_full_workflow_test', 'panel'))

In [13]:
# to use the DLS and not the panel set this to None
panelNames = None
#panelNames = glob.glob(os.path.join('data', '20210407_duml_full_workflow_test', 'panel', 'IMG_0008*.tif'))
#blocked_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sun_blocked')
#surface_dir = os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'sea')
sky_dir = os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'sky')

warp_img_dir = os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'sky')

#sunlight_blocked_out = process_micasense_subset(blocked_dir, panelNames, warp_img_dir, overwrite=False)
#surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)
sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)

Not using panel irradiance.
used warp dir data/20210304_duml_newgimbal_mapping_testing/sky
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 2
Finished aligning band 4
Finished aligning band 1
Finished aligning band 3
Finished aligning band 0
Saving time: 0:06:36.121410
Alignment+Saving rate: 0.32 images per second
Finished saving images.
exiftool -csv="data/20210304_duml_newgimbal_mapping_testing/sky/stacks/log.csv" -overwrite_original data/20210304_duml_newgimbal_mapping_testing/sky/stacks
Finished saving image metadata.


In [14]:
! exiftool -csv="data/20210304_duml_newgimbal_mapping_testing/sky/stacks/log.csv" -overwrite_original data/20210304_duml_newgimbal_mapping_testing/sky/stacks

    1 directories scanned
  128 image files updated


In [11]:
# to use the DLS and not the panel set this to None
panelNames = None
#panelNames = glob.glob(os.path.join('data', '20210407_duml_full_workflow_test', 'panel', 'IMG_0008*.tif'))
#blocked_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sun_blocked')
surface_dir = os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'sea')
sky_dir = os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'sky')

warp_img_dir = os.path.join('data', '20210304_duml_newgimbal_mapping_testing', 'sea')

#sunlight_blocked_out = process_micasense_subset(blocked_dir, panelNames, warp_img_dir, overwrite=False)
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)
sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)

Not using panel irradiance.
used warp dir data/20210304_duml_newgimbal_mapping_testing/sea
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 2
Finished aligning band 0
Finished aligning band 3
Finished aligning band 1
Finished aligning band 4
Saving time: 0:34:53.511129
Alignment+Saving rate: 0.31 images per second
Finished saving images.
exiftool -csv="data/20210304_duml_newgimbal_mapping_testing/sea/stacks/log.csv" -overwrite_original data/20210304_duml_newgimbal_mapping_testing/sea/stacks
Finished saving image metadata.
Not using panel irradiance.


ValueError: Provided file path does not have a band name: data/20210304_duml_newgimbal_mapping_testing/sea/stacks/ssNzzmZLkRg4Kffm2g8N.tif

In [12]:
! exiftool -csv="data/20210304_duml_newgimbal_mapping_testing/sea/stacks/log.csv" -overwrite_original data/20210304_duml_newgimbal_mapping_testing/sea/stacks

    1 directories scanned
  645 image files updated


In [24]:
surface_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sea')
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/20210407_duml_full_workflow_test/sea
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 2
Finished aligning band 1
Finished aligning band 0
Finished aligning band 4
Finished aligning band 3
Saving time: 0:10:09.499021
Alignment+Saving rate: 0.31 images per second
Finished saving images.
exiftool -csv="data/20210407_duml_full_workflow_test/sea/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sea/stacks
Finished saving image metadata.


In [25]:
! exiftool -csv="data/20210407_duml_full_workflow_test/sky/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sky/stacks

    1 directories scanned
  144 image files updated


In [26]:
! exiftool -csv="data/20210407_duml_full_workflow_test/sea/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sea/stacks

    1 directories scanned
  187 image files updated


Now process the RedEdge Images

In [28]:
# to use the DLS and not the panel set this to None
panelNames = None
panelNames = glob.glob(os.path.join('data', '20210407_duml_full_workflow_test', 'panel_rededge', 'IMG_0003*.tif'))
blocked_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sunblocked_rededge')
sky_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sky_rededge')
surface_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sea_rededge')

warp_img_dir = os.path.join('data', '20210407_duml_full_workflow_test', 'sea_rededge')

sunlight_blocked_out = process_micasense_subset(blocked_dir, panelNames, warp_img_dir, overwrite=False)
sky_out = process_micasense_subset(sky_dir, panelNames, warp_img_dir, overwrite=False)
surface_out = process_micasense_subset(surface_dir, panelNames, warp_img_dir, overwrite=False)

Panel irradiance calculated.
used warp dir data/20210407_duml_full_workflow_test/sea_rededge
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 2
Finished aligning band 0
Finished aligning band 4
Finished aligning band 3
Finished aligning band 1
Saving time: 0:00:15.060539
Alignment+Saving rate: 0.66 images per second
Finished saving images.
exiftool -csv="data/20210407_duml_full_workflow_test/sunblocked_rededge/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sunblocked_rededge/stacks
Finished saving image metadata.
Panel irradiance calculated.
used warp dir data/20210407_duml_full_workflow_test/sea_rededge
Aligning images. Depending on settings this can take from a few seconds to many minutes
Finished aligning band 2
Finished aligning band 0
Finished aligning band 1
Finished aligning band 3
Finished aligning band 4
Saving time: 0:00:04.473115
Alignment+Saving rate: 0.89 images per second
Finished sav

In [29]:
! exiftool -csv="data/20210407_duml_full_workflow_test/sunblocked_rededge/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sunblocked_rededge/stacks

    1 directories scanned
   10 image files updated


In [30]:
! exiftool -csv="data/20210407_duml_full_workflow_test/sky_rededge/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sky_rededge/stacks

    1 directories scanned
    4 image files updated


In [31]:
! exiftool -csv="data/20210407_duml_full_workflow_test/sea_rededge/stacks/log.csv" -overwrite_original data/20210407_duml_full_workflow_test/sea_rededge/stacks

    1 directories scanned
    9 image files updated


In [51]:
skyimgset = imageset.ImageSet.from_directory(sky_dir)
skyOutputPath = os.path.join(sky_dir,'..','skystacks')
fullCsvPath = write_exif_csv(skyimgset, skyOutputPath)
write_img_exif(fullCsvPath, skyOutputPath)

exiftool -csv="data/170321_duml_testing/360_test/sky/../skystacks/log.csv" -overwrite_original data/170321_duml_testing/360_test/sky/../skystacks


True

In [52]:
! exiftool -csv="data/170321_duml_testing/360_test/sky/../skystacks/log.csv" -overwrite_original data/170321_duml_testing/360_test/sky/../skystacks

    1 directories scanned
   62 image files updated


In [53]:
seaimgset = imageset.ImageSet.from_directory(sea_dir)
seaOutputPath = os.path.join(sea_dir,'..','seastacks')
fullCsvPath = write_exif_csv(seaimgset, seaOutputPath)
write_img_exif(fullCsvPath, seaOutputPath)

exiftool -csv="data/170321_duml_testing/360_test/sea/../seastacks/log.csv" -overwrite_original data/170321_duml_testing/360_test/sea/../seastacks


True

In [54]:
! exiftool -csv="data/170321_duml_testing/360_test/sea/../seastacks/log.csv" -overwrite_original data/170321_duml_testing/360_test/sea/../seastacks

    1 directories scanned
  317 image files updated


In [36]:
! exiftool -csv="data/170321_duml_testing/360_test/sky/../skystacks/log.csv" -overwrite_original data/170321_duml_testing/360_test/sky/../skystacks

    1 directories scanned
   62 image files updated


In [37]:
! exiftool -csv="data/170321_duml_testing/360_test/sea/../seastacks/log.csv" -overwrite_original data/170321_duml_testing/360_test/sea/../seastacks

    1 directories scanned
  317 image files updated
